In [16]:
from PIL import Image, ImageDraw
import pytesseract

def process_image(input_path, output_path):
    # Open the image file
    img = Image.open(input_path)

    # Get the text and bounding box information from the image using Tesseract OCR
    result = pytesseract.image_to_boxes(img, config='--psm 11')

    # Print bounding box information
    print("Bounding box information:")
    print(result)

    # Create an image with the same size as the input image
    output_img = img.copy()
    draw = ImageDraw.Draw(output_img)

    # Get image height for coordinate transformation
    img_height = img.size[1]

    # Iterate through each character and draw a bounding box
    for char_info in result.splitlines():
        char_data = char_info.split()
        char = char_data[0]

        if char.isalpha():
            # Get the bounding box for the character
            bbox = [int(coord) for coord in char_data[1:5]]

            # Transform coordinates to PIL coordinate system
            bbox[1], bbox[3] = img_height - bbox[3], img_height - bbox[1]

            # Draw a rectangle around the character in the output image with red color
            draw.rectangle(bbox, outline='red')

    # Save the output image
    output_img.save(output_path)

    # Return the array of letters
    letters = [char.upper() for char in result if char.isalpha()]
    return letters

if __name__ == "__main__":
    input_image_path = r"supervised\papyrus_images\papyrus-001.jpg"
    output_image_path = r"boundingboxes\papyrus-001.jpg"

    result = process_image(input_image_path, output_image_path)
    print("Array of letters:", result)


Bounding box information:
N 44 119 74 154 0
o 69 119 92 154 0
s 76 123 129 140 0
e 113 119 132 154 0
m 132 124 172 140 0
a 153 119 172 154 0

Array of letters: ['N', 'O', 'S', 'E', 'M', 'A']


In [ ]:
input_image_path = r"supervised\century_schoolbook_images\century_schoolbook-011.jpg"
output_image_path = r"boundingboxes\cent-011.jpg"
